# AIS421: Lecture 02: Prompt Engineering
## Instructor: Dr. Ensaf Hussein
## University: Nile University
## Semester: Spring 2026

This notebook provides a comprehensive introduction to prompt engineering with hands-on examples.

In [ ]:
# Save this notebook as "prompt_engineering_crash_course.ipynb"
# You can run cells sequentially to learn prompt engineering concepts

# Install required packages
!pip install openai

import os
from openai import OpenAI
import getpass

## Part 1 — Quick Introduction

### What is Prompt Engineering?
A discipline focused on developing and optimizing prompts to effectively interact with Large Language Models (LLMs). It is a hybrid between programming, instructing, and teaching.

### Why It Matters
- **Research & Discovery**: Helps test and evaluate the boundaries and limitations of LLMs
- **Innovation**: Enables complex applications like autonomous agents and reasoning systems
- **Career Path**: Companies like Anthropic now hire "Prompt Engineers & Librarians" with competitive salaries ($250k - $335k)

## Part 2 — LLM Settings & Configurations

Configurable parameters that influence how an LLM generates responses.

### 1. Main Definitions

**Temperature**: Controls randomness
- **Low (e.g., 0.2)**: More predictable, fact-based responses
- **High (e.g., 0.8)**: More creative, varied responses

**Top P (Nucleus Sampling)**: Determines the range of possible next words
- **Low (e.g., 0.1)**: More confident and exact responses
- **High (e.g., 0.9)**: More diverse and exploratory

> **Tip**: Adjust either Temperature or Top P, but not both.

### 2. Output Length & Structure
- **Max Length**: Maximum number of tokens (words/subwords) the model can generate
- **Stop Sequences**: Specific characters that signal the model to stop
  - *Example*: Adding "11." as a stop sequence stops a numbered list at 10 items

### 3. Repetition Control
- **Frequency Penalty**: Penalizes overused tokens. Higher values result in more varied vocabulary
- **Presence Penalty**: Encourages the model to use new words/topics. Higher values result in more diverse wording

> **Tip**: Adjust either Frequency or Presence Penalty, but not both.

In [ ]:
# Setup OpenAI client
print("Please enter your OpenAI API key:")
api_key = getpass.getpass()

client = OpenAI(api_key=api_key)

def get_completion(prompt, temp=0.7, top_p=1.0, max_tokens=256,
                   frequency_penalty=0, presence_penalty=0,
                   stop_sequences=None):
    """
    A flexible function to get completions from OpenAI with various parameters.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Using a cost-effective model
        messages=[{"role": "user", "content": prompt}],
        temperature=temp,
        top_p=top_p,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop_sequences
    )
    return response.choices[0].message.content

## Part 3 — Your First Prompt (Hands-On)

In [ ]:
# Basic completion example
print("Example 1: Basic completion with default settings")
result = get_completion("The sky is")
print(f"Prompt: 'The sky is'\nOutput: '{result}'\n")

# Compare temperature effects
print("Example 2: Comparing temperature effects")
print("\nLow temperature (0.2) - More predictable:")
result_low = get_completion("Write a one-sentence story about a robot", temp=0.2)
print(result_low)

print("\nHigh temperature (0.9) - More creative:")
result_high = get_completion("Write a one-sentence story about a robot", temp=0.9)
print(result_high)

## Part 4 — Core Techniques (By Example)

### 1. Zero-Shot Prompting
Asking a question without providing any examples.

In [ ]:
print("=== Zero-Shot Prompting ===\n")
prompt = "What is prompt engineering? Explain in one sentence."
result = get_completion(prompt, temp=0.3)
print(f"Prompt: {prompt}\n")
print(f"Output: {result}")

### 2. Few-Shot Prompting
Providing demonstrations to guide the model.

In [ ]:
print("\n=== Few-Shot Prompting ===\n")
prompt = """Classify the sentiment as Positive or Negative:

This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //"""

result = get_completion(prompt, temp=0.2)
print(f"Prompt with examples:\n{prompt}\n")
print(f"Output: {result}")

### 3. Instruction Prompting
Providing specific constraints and formatting.

In [ ]:
print("\n=== Instruction Prompting ===\n")
prompt = """Extract the names of places from the text below.
Format: Place: <comma_separated_list>

Text: 'at the Champalimaud Centre in Lisbon.'"""

result = get_completion(prompt, temp=0.2)
print(f"Prompt: {prompt}\n")
print(f"Output: {result}")

## Part 5 — Advanced Techniques

### 1. Chain-of-Thought (CoT)
Encourages step-by-step reasoning to solve logic traps.

In [ ]:
print("=== Chain-of-Thought Prompting ===\n")

# Without CoT
prompt_direct = "A juggler can juggle 16 balls. Half are golf balls, and half of the golf balls are blue. How many blue golf balls?"
print("Without CoT (Direct answer):")
result_direct = get_completion(prompt_direct, temp=0.2)
print(f"Q: {prompt_direct}")
print(f"A: {result_direct}\n")

# With CoT
prompt_cot = "A juggler can juggle 16 balls. Half are golf balls, and half of the golf balls are blue. How many blue golf balls? Let's think step by step."
print("With CoT (Step-by-step reasoning):")
result_cot = get_completion(prompt_cot, temp=0.2)
print(f"Q: {prompt_cot}")
print(f"A: {result_cot}")

### 2. Program-Aided Language Models (PAL)
Offloads math to a Python interpreter to ensure calculation accuracy.

In [ ]:
print("\n=== Program-Aided Language Models (PAL) ===\n")

prompt_pal = """The bakers baked 200 loaves. They sold 93 in the morning and 39 in the afternoon. 6 loaves were returned unsold. How many loaves are left?

Let's solve this step by step with Python code:"""

result_pal = get_completion(prompt_pal, temp=0.2, max_tokens=300)
print(f"Prompt: {prompt_pal}\n")
print(f"Output with code:\n{result_pal}")

### 3. ReAct (Reason + Act)
Interleaves reasoning traces with tool-use actions.

In [ ]:
print("\n=== ReAct Prompting (Reason + Act) ===\n")

prompt_react = """Answer the following question using the ReAct format:
Question: What device can control the Front Row media center besides the original Apple Remote?

Thought 1: I need to search for what the Apple Remote was originally designed for.
Action 1: Search[Apple Remote original purpose]
Observation 1: The Apple Remote was originally designed to control the Front Row media center on Mac computers.

Thought 2: Now I need to find what other devices can control Front Row.
Action 2: Search[devices compatible with Front Row]
Observation 2: Front Row could also be controlled by some third-party universal remotes and iPhone apps with Wi-Fi remote capabilities.

Thought 3: I have enough information to answer the question.
Action 3: Finish[Third-party universal remotes and iPhone apps with Wi-Fi remote capabilities can control Front Row besides the original Apple Remote.]

Now try this format with: What is the capital of France?"""

result_react = get_completion(prompt_react, temp=0.3, max_tokens=400)
print(result_react)

### 4. Generated Knowledge Prompting
Generate facts first to improve commonsense reasoning.

In [ ]:
print("\n=== Generated Knowledge Prompting ===\n")

prompt_knowledge = """First, generate relevant facts about golf, then answer the question.

Question: Is the statement true? 'Part of golf is trying to get a higher point total than others.'

Step 1 - Generate facts about golf:"""
result_knowledge = get_completion(prompt_knowledge, temp=0.4, max_tokens=400)
print(result_knowledge)

## Part 6 — Best Practices & Cheat Sheet

### Key Principles

| Principle | Description | Example |
|-----------|-------------|---------|
| **Be Specific** | Use clear action words | "Write a summary," "Classify as," "Extract dates" |
| **Focus on "Do"** | Tell what to do, not what not to do | ✅ "Write in English" vs ❌ "Don't write in French" |
| **Use Separators** | Mark boundaries clearly | "### Instructions ###\nSummarize:\n### Text ###\n{text}" |
| **Iterate** | Start simple, add complexity gradually | Start with zero-shot → Add examples → Add constraints |

In [ ]:
print("=== Putting It All Together: Best Practices Demo ===\n")

# Example of following best practices
prompt_best = """### TASK ###
Summarize the following product review in 2-3 sentences, focusing only on pros and cons.

### FORMAT ###
Pros: <list pros>
Cons: <list cons>
Summary: <brief summary>

### REVIEW ###
I bought this wireless mouse last week. The battery life is amazing - been using it daily and still at 100%. It's very comfortable for my hand size. However, the Bluetooth connection sometimes drops randomly, especially when my laptop is more than 3 feet away. The silent clicks are great for working in libraries, but the scroll wheel feels a bit loose. Customer support was responsive when I emailed them about the connection issues.

### RESPONSE ###"""

result_best = get_completion(prompt_best, temp=0.3)
print(f"Prompt following best practices:\n{result_best}")

# Quick comparison: Poor vs Good prompting
print("\n=== Poor Prompting vs Good Prompting ===\n")

poor_prompt = "tell me about ai"
good_prompt = """### INSTRUCTION ###
Provide a comprehensive overview of Artificial Intelligence (AI) with:
1. A simple definition (1 sentence)
2. Three main types of AI
3. Two real-world applications
4. One current limitation

Keep the total response under 150 words.
### END ###"""

print("Poor prompt: 'tell me about ai'")
print("Good prompt includes:\n- Clear structure\n- Specific requirements\n- Format constraints\n- Word limit\n")

## Practice Exercises

In [ ]:
print("=== Practice Exercises ===\n")
print("Try modifying the following exercises to test your skills:\n")

exercises = [
    "1. Zero-shot: Ask the model to classify movie genres",
    "2. Few-shot: Provide 3 examples of English to Spanish translations, then translate 'Hello, how are you?",
    "3. Chain-of-Thought: Solve a multi-step math word problem",
    "4. Instruction-based: Extract email addresses and phone numbers from a text",
    "5. Format control: Generate a response in JSON format"
]

for ex in exercises:
    print(ex)

print("\nExample exercise to try:")
prompt_exercise = """Translate these English phrases to French:

Example 1: Good morning -> Bonjour
Example 2: Thank you -> Merci
Example 3: How much? -> Combien?

Now translate: Where is the nearest restaurant?"""

print(f"\nExercise prompt:\n{prompt_exercise}")
print("\nRun this cell to see the result:")
# Uncomment to test:
# result_exercise = get_completion(prompt_exercise, temp=0.3)
# print(result_exercise)

## Summary & Key Takeaways

In [ ]:
print("""
╔════════════════════════════════════════════════════════════╗
║                 PROMPT ENGINEERING CHEAT SHEET             ║
╠════════════════════════════════════════════════════════════╣
║                                                            ║
║   Start Simple → Iterate & Refine                          ║
║   Be Specific & Clear                                      ║
║   Adjust Parameters Strategically                          ║
║   Use Examples for Complex Tasks                           ║
║   Offload Computation When Needed                          ║
║   Think Step-by-Step for Reasoning                         ║
║                                                            ║
║  Remember:                                                 ║
║  • Temperature = Creativity                                ║
║  • Top-P = Word Choice Range                               ║
║  • Max Tokens = Response Length                            ║
║  • Stop Sequences = Control Points                         ║
║                                                            ║
╚════════════════════════════════════════════════════════════╝
""")

print("\n---")
print("**Next Steps:**")
print("1. Practice with different prompts")
print("2. Experiment with parameters")
print("3. Try combining techniques")
print("4. Build your own prompt templates")
print("5. Explore more advanced topics like Auto-CoT and Tree of Thoughts")